In [18]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import time as tm
from typing import List 
from dataclasses import dataclass
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [19]:
x = sp.Symbol('x')
y = sp.lambdify(x, sp.diff(x*sp.exp(x), x))
y(1)

5.43656365691809

In [20]:
class Function:
    def __init__(self, f):
        x = sp.Symbol('x')
        self.f = sp.lambdify(x, f)
        self.g = sp.lambdify(x, f.diff(x))
        self.h = sp.lambdify(x, f.diff(x, 2))
    
    def calculate(self, x):
        return self.f(x)
    
    def gradient(self, x):
        return self.g(x)
    
    def hesse(self, x):
        return self.h(x)

In [21]:
@dataclass
class State:
    it: int
    x: float
    y: float

@dataclass
class Result:
    point: float = None
    value: float = None
    solution_time: float = 0
    iteration_amount: float = None
    progress: List[State] = None
    status: str = ''

In [22]:
def armijo_line_search(f:Function, x):
    a = 0.5
    step = 1
    c = 1e-4
    p = - f.gradient(x)
    while not f.calculate(x + step * p) <= f.calculate(x) + c * step * f.gradient(x) * p:
        step *= a
    return step

In [35]:
def inexact_search(f:Function, a, b, tol:float, max_it=1000):
    start_time = tm.time()
    res = Result()
    x = np.random.uniform(a, b)
    p = -f.gradient(x)
    it = 0
    res.progress = []
    res.progress.append([it, x, f.calculate(x)])
    print('It: {}, \t x = {:.3f}, f = {:.3f}'.format(it, x, f.calculate(x)))
    while np.abs(p) >= tol and it < max_it:
        step = armijo_line_search(f, x)
        x += step * p
        p = -f.gradient(x)
        it += 1
        res.progress.append([it, x, f.calculate(x)])
        print('It: {}, \t x = {:.3f}, f = {:.3f}'.format(it, x, f.calculate(x)))
    if it == max_it:
        print('Failure')
        res.status = 'Failure'
    else:
        print('\nSolution: \t x = {:.3f}, f = {:.3f}'.format(x, f.calculate(x)))
        res.status = 'Success'
    res.point = x
    res.value = f.calculate(x)
    res.solution_time = tm.time() - start_time
    res.iteration_amount = it
    return res

In [36]:
def plot_func(f:Function, x, a, b):
    i = np.linspace(a, b, 100)
    plt.plot(i, f.calculate(i))
    plt.plot(x, f.calculate(x), 'ro')
    plt.xlabel('$x$')
    plt.ylabel('$f(x)$')
    plt.title('Objective Function')

In [37]:
def plot_path(f:Function, x):
    plot_func(f, )

In [38]:
a = inexact_search(Function(x**2), -1, 10, 0.1)

b = inexact_search(Function(x**2), -1, 10, 0.1)


print(a.progress)
print(b.progress)


It: 0, 	 x = 3.286, f = 10.801
It: 1, 	 x = 0.000, f = 0.000

Solution: 	 x = 0.000, f = 0.000
It: 0, 	 x = 5.231, f = 27.363
It: 1, 	 x = 0.000, f = 0.000

Solution: 	 x = 0.000, f = 0.000
[[0, 3.2864223410318463, 10.800571803633241], [1, 0.0, 0.0]]
[[0, 5.231012399063307, 27.36349071915405], [1, 0.0, 0.0]]


In [41]:
c = inexact_search(Function((x+5)**4), -6, 2, 0.1)
print(c.progress)

It: 0, 	 x = -2.098, f = 70.919
It: 1, 	 x = -5.153, f = 0.001

Solution: 	 x = -5.153, f = 0.001
[[0, -2.098041641240866, 70.9193432568734], [1, -5.1528469868626345, 0.0005457924539264545]]


In [128]:
inexact_search(Function(x*exp(x)), -2, 0, 0.1)

It: 1, 	 x = -0.898, f = -0.366

Solution: 	 x = -0.898, f = -0.366


-0.3658225684164918

In [124]:
inexact_search(Function(x*exp(-x)), -2, 6, 0.1)

It: 1, 	 x = 2.528, f = 0.202
It: 2, 	 x = 2.650, f = 0.187
It: 3, 	 x = 2.766, f = 0.174
It: 4, 	 x = 2.877, f = 0.162
It: 5, 	 x = 2.983, f = 0.151
It: 6, 	 x = 3.083, f = 0.141

Solution: 	 x = 3.083, f = 0.141


0.14123951855747224